In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import logging
import math
import time
import statistics as stats
import json
from datetime import datetime, timedelta
import os, sys
import matplotlib.pyplot as plt
import glob
from scipy.signal import medfilt, butter, filtfilt, lfilter, find_peaks, find_peaks_cwt,resample, detrend
from scipy.signal import welch, spectrogram, get_window
from scipy.stats import linregress
from scipy import stats
from scipy.interpolate import interp1d

Goal is to create a feature set consisting of

1. Fractal dimension
2. VMS
3. LI

OUTPUT: A csv file of the features with 0 for normal and 1 for pathological


In [2]:
cwd = os.getcwd()
print(cwd)
#os.chdir(cwd+'/ML')

/home/mike/Desktop/apps/research/VAG/notebooks/ML


In [3]:
healthy_dir = "../../data/open_vag/normal/"
pathology_dir = "../../data/open_vag/pathology/"
#healthy_base = "novag"
#pathology_base  = "abvag"

healthy_subs = [] # 51
pathology_subs = [] # 38

exclude = {".ipynb_checkpoints"}

healthy_files = [f for f in os.listdir(healthy_dir) if '.ipynb_checkpoints' not in f]
print(f" healthy vag signals {len(healthy_files)}")
pathology_files = [f for f in os.listdir(pathology_dir) if '.ipynb_checkpoints' not in f]
print(f" pathology vag signals {len(pathology_files)}")

max_size_files = max(len(healthy_files), len(pathology_files))

for i in range(0, max_size_files):
    if(i < len(healthy_files)):
        # read in a healthy_file
        file_path = os.path.join(healthy_dir, healthy_files[i])
        with open(file_path, 'r') as file:
            # Read all the floating-point numbers into a list
            vags = [float(x) for x in file.read().split()]
            healthy_subs.append(vags)
    if(i < len(pathology_files)):
        file_path = os.path.join(pathology_dir, pathology_files[i])
        with open(file_path, 'r') as file:
            # Read all the floating-point numbers into a list
            vags = [float(x) for x in file.read().split()]
            pathology_subs.append(vags)   

 healthy vag signals 51
 pathology vag signals 38


First step is to normalise, filter, resample and split the data sets

In [4]:
def amplitude_normal(signal):
    sig = np.array(signal)
    data_min = min(sig)
    data_max = max(sig)
    normal = (sig - data_min) / (data_max - data_min)
    return normal.tolist()

# not used anymore
def resample_signal(signal, points, fs):
    t = np.linspace(0, fs, num=len(signal))
    x_new = np.linspace(t.min(), t.max(), points)
    interpolator = interp1d(t, signal, kind='linear')
    y_new = interpolator(x_new)
    return y_new

def split_signal(signal):
    middle = len(signal) // 2
    # Split the array into two halves
    first_half = signal[:middle]
    second_half = signal[middle:]
    
    return [first_half, second_half]

def build_filter(frequency, sample_rate, filter_type, filter_order):
    #nyq = 0.5 * sample_rate
    if filter_type == "bandpass":
        #nyq_cutoff = (frequency[0] / nyq, frequency[1] / nyq)
        b, a = butter(filter_order, (frequency[0], frequency[1]), btype=filter_type, analog=False, output='ba', fs=sample_rate)
    elif filter_type == "low":
        #nyq_cutoff = frequency[1] / nyq
        b, a = butter(filter_order, frequency[1] / ny, btype=filter_type, analog=False, output='ba', fs=sample_rate)
    elif filter_type == "high":
        #nyq_cutoff = frequency[0] / nyq
        b, a = butter(filter_order, frequency[0], btype=filter_type, analog=False, output='ba', fs=sample_rate)

    return b, a

def filter_signal(b, a, signal, filter):
    if(filter=="lfilter"):
        return lfilter(b, a, signal)
    elif(filter=="filtfilt"):
        return filtfilt(b, a, signal)
    elif(filter=="sos"):
        return sosfiltfilt(sos, signal)

r_l = 8000
fs = 2000 # 2khz sampling rate
low_cut_off = 100 # removes muscle artifacts and baseline wander
high_cut_off = 750
filter_order = 5   # 9th order has been used in literature?
filter_type =  "bandpass"  #"bandpass", high, low
b,a = build_filter((low_cut_off, high_cut_off), fs, filter_type, filter_order)

n_res = [] # resampled normal vag signal
n_ext_flex = [] # resampled split vag signal
p_res = [] # resampled normal pathological signal
p_ext_flex = [] # resampled split pathological signal

for d_set in healthy_subs:
    n = amplitude_normal(d_set)
    #n_resampled = resample_signal(n, r_l, fs)
    y_new = resample(n, r_l)
    y_new_f = filter_signal(b,a,y_new,"filtfilt")
    n_res.append(y_new)

    # split into extension / flexion
    ext_flex = split_signal(y_new_f)  # ext is 0 and flex is 1
    n_ext_flex.append(ext_flex)

for d_set in pathology_subs:
    p = amplitude_normal(d_set)
    #p_resampled = resample_signal(p, r_l, fs)
    y_new = resample(p, r_l)
    y_new_f = filter_signal(b,a,y_new,"filtfilt")
    p_res.append(y_new_f)

    # split into extension / flexion
    ext_flex = split_signal(y_new_f)  # ext is 0 and flex is 1
    p_ext_flex.append(ext_flex)

print(len(n_res[0]), len(n_ext_flex[0][0]), len(n_ext_flex[0][1]))

8000 4000 4000


In [5]:
# compute the fractal dimension of each signal 

def apply_hanning(d):
    hanning_window = np.hanning(len(d))
    windowed_signal = d * hanning_window
    return windowed_signal

# assumes len(data) is a fft multiple i.e 4096 or 8192
def compute_fft_mag(data):
    fftpoints = len(data) #int(math.pow(2, math.ceil(math.log2(len(data)))))
    fft = np.fft.fft(data, n=fftpoints)
    mag = np.abs(fft) #/ (fftpoints/2)
    return mag

# assumes a numpy arrray
def compute_power_spectrum(fft_mag):
    power = np.square(fft_mag)
    return power
    
def compute_fd(slope):
    fd = (5 - abs(slope)) / 2
    return fd

def vag_to_fd(s):
    s_han = apply_hanning(s)
    #compute fft and power spectrum
    s_fft = compute_fft_mag(s_han)
    s_pwr = compute_power_spectrum(s_fft)
    # get the frequency axis
    s_x = np.fft.fftfreq(len(s), d=1/fs)
    # compute the log log
    s_log_f = np.log10(s_x)
    s_log_pwr = np.log10(s_pwr)

    indices = np.where((s_x > 10) & (s_x < 300))[0]
    x = s_log_f[indices]
    y = s_log_pwr[indices]
    m, b, r_value, p_value, std_err = linregress(x, y)
    # we only care about the slope for fd
    s_fd = compute_fd(m)
    return s_fd

# frequency range to compute fd (Hz)
min_f = 100  
max_f = 750

n_fd = []
n_ext_flex_fd = []
p_fd = []
p_ext_flex_fd = []

# normal vag signals
for i, n in enumerate(n_res):
    fd = vag_to_fd(n)
    n_fd.append(fd)

    # compute fd for extension and flexion
    fd_ext = vag_to_fd(n_ext_flex[i][0])
    fd_flex = vag_to_fd(n_ext_flex[i][1])
    n_ext_flex_fd.append([fd_ext, fd_flex])

    # compute for pathological signal
    if(i < len(p_res)):
        fd = vag_to_fd(p_res[i])
        p_fd.append(fd)

        fd_ext = vag_to_fd(p_ext_flex[i][0])
        fd_flex = vag_to_fd(p_ext_flex[i][1])
        p_ext_flex_fd.append([fd_ext, fd_flex])

print(len(n_fd), len(p_fd))

print(f"avg fd normal: {float(np.mean(n_fd))} +/- {round(np.std(n_fd),4)}")
print(f"avg fd pathol: {np.mean(p_fd)} +/- {round(np.std(p_fd),4)}")

51 38
avg fd normal: 1.8476369324826152 +/- 0.2277
avg fd pathol: -2.4423922863813936 +/- 0.1925


/tmp/ipykernel_7700/456299692.py:32: RuntimeWarning: divide by zero encountered in log10
  s_log_f = np.log10(s_x)
/tmp/ipykernel_7700/456299692.py:32: RuntimeWarning: invalid value encountered in log10
  s_log_f = np.log10(s_x)


statistically highly signiﬁcant (p < 0.01),
statistically signiﬁcant (0.01 < p < 0.05).

In [6]:
# test p values for fd
# check p values agains the paper - IDENTICAL
fd_t, fd_p_value = stats.ttest_ind(n_fd, p_fd)
print("FD P-value:", fd_p_value)

#extenstion
n_fd1 = []
p_fd1 = []
for r in n_ext_flex_fd:
    n_fd1.append(r[0])
for r in p_ext_flex_fd:
    p_fd1.append(r[0])

fd1_t, fd1_p_value = stats.ttest_ind(n_fd1, p_fd1)
print("FD1 P-value:", fd1_p_value)

n_fd2 = []
p_fd2 = []
for r in n_ext_flex_fd:
    n_fd2.append(r[1])
for r in p_ext_flex_fd:
    p_fd2.append(r[1])

fd2_t, fd2_p_value = stats.ttest_ind(n_fd2, p_fd2)
print("FD2 P-value:", fd2_p_value)


FD P-value: 8.901377374682255e-89
FD1 P-value: 0.6043349254950423
FD2 P-value: 0.2988447806655629


In [7]:
# compute the VMS -> Mean Squared every 5ms 
# 2khz and 5ms = 10 samples per block

# window in seconds - 0.005 is 5ms fs in Hz 
def compute_mean_squared_variance(signal, fs, window=0.005):
    samples_per_group = int(window * fs)
    num_groups = len(signal) // samples_per_group
    
    # Slice the array to only include full groups
    trimmed_array = signal[:num_groups * samples_per_group]
    
    # Reshape the array into a 2D array where each row is a group of 10 samples
    reshaped_array = trimmed_array.reshape(num_groups, samples_per_group)
    #mean = np.mean(reshaped_array, axis=1)
    mean_squared = [np.mean(row)**2 for row in reshaped_array]
    variance = np.var(mean_squared)
    
    return variance, mean_squared
    
n_list = []
p_list = []
n_mean_sqr = []
n_mean_sqr_ext_flex = []
p_mean_sqr = []
p_mean_sqr_ext_flex = []

for i, n in enumerate(n_res):
    var, n_m_sqr = compute_mean_squared_variance(n, fs)
    n_mean_sqr.append(var)
    n_list.append(n_m_sqr)
    var_ext, n_sqr_ext = compute_mean_squared_variance(n_ext_flex[i][0], fs)
    var_flex, n_sqr_flx = compute_mean_squared_variance(n_ext_flex[i][1], fs)
    n_mean_sqr_ext_flex.append([var_ext, var_flex])

    if(i < len(p_res)):
        p_var, p_m_sqr = compute_mean_squared_variance(p_res[i], fs)
        p_mean_sqr.append(p_var)
        p_list.append(p_m_sqr)
    
        p_var_ext, p_sqr_ext = compute_mean_squared_variance(p_ext_flex[i][0], fs)
        p_var_flex, p_sqr_flx = compute_mean_squared_variance(p_ext_flex[i][1], fs)
        p_mean_sqr_ext_flex.append([p_var_ext, p_var_flex])


print(len(n_mean_sqr), len(p_mean_sqr))

print(f"VMS normal: {float(np.mean(n_mean_sqr))} +/- {round(np.std(n_mean_sqr),4)}")
print(f"VMS pathol: {np.mean(p_mean_sqr)} +/- {round(np.std(p_mean_sqr),4)}")


51 38
VMS normal: 0.005747573490845162 +/- 0.0061
VMS pathol: 1.332847933043984e-07 +/- 0.0


In [8]:
# check p values agains the paper - IDENTICAL
vms_t, vms_p_value = stats.ttest_ind(n_mean_sqr, p_mean_sqr)
print("VMS P-value:", vms_p_value)

#extenstion
n_vms1 = []
p_vms1 = []
for r in n_mean_sqr_ext_flex:
    n_vms1.append(r[0])
for r in p_mean_sqr_ext_flex:
    p_vms1.append(r[0])

vms1_t, vms1_p_value = stats.ttest_ind(n_vms1, p_vms1)
print("VMS1 P-value:", vms1_p_value)

#flextion
n_vms2 = []
p_vms2 = []
for r in n_mean_sqr_ext_flex:
    n_vms2.append(r[1])
for r in p_mean_sqr_ext_flex:
    p_vms2.append(r[1])
    
vms2_t, vms2_p_value = stats.ttest_ind(n_vms2, p_vms2)
print("VMS1 P-value:", vms2_p_value)


VMS P-value: 1.5533644595748518e-07
VMS1 P-value: 0.21970206938834727
VMS1 P-value: 0.36684764048447094


In [9]:
# compute li over full signals

def compute_fft_mag_positive_f(data, fftpoints):
    fft = np.fft.fft(data, n=fftpoints)
    mag = np.abs(fft) / (fftpoints/2)
    return mag

def compute_loading_intensity(s, fft_points, fs, high_cut_off):
    LI = 0
    fc = high_cut_off
    kc = int((fft_points/fs)* fc) + 1

    magnitudes = compute_fft_mag_positive_f(s, fft_points)

    f = []
    for i in range(0, int(fft_points/2)+1):
        f.append((fs*i)/fft_points)

    for k in range(0, kc):
        LI = LI + (magnitudes[k] * f[k])

    return LI

n_li = []
n_li_ext_flex = []
p_li = []
p_li_ext_flex = []

f_cut_off = 300 # as per the FD

for i, n in enumerate(n_res):
    li = compute_loading_intensity(n, len(n), fs, 1000) # high cutoff is nyquist
    n_li.append(li)
    n_li_ext = compute_loading_intensity(n_ext_flex[i][0], len(n_ext_flex[i][0]), fs, f_cut_off)
    n_li_flx = compute_loading_intensity(n_ext_flex[i][1], len(n_ext_flex[i][1]), fs, f_cut_off)
    n_li_ext_flex.append([n_li_ext, n_li_flx])

    if(i < len(p_res)):
        li = compute_loading_intensity(p_res[i], len(p_res[i]), fs, 1000) # high cutoff is nyquist
        p_li.append(li)
        p_li_ext = compute_loading_intensity(p_ext_flex[i][0], len(p_ext_flex[i][0]), fs, f_cut_off)
        p_li_flx = compute_loading_intensity(p_ext_flex[i][1], len(p_ext_flex[i][1]), fs, f_cut_off)
        p_li_ext_flex.append([p_li_ext, p_li_flx])

print(len(n_li_ext_flex), len(p_li_ext_flex))

print(f"LI normal: {float(np.mean(n_li))} +/- {round(np.std(n_li),4)}")
print(f"LI pathol: {np.mean(p_li)} +/- {round(np.std(p_li),4)}")

51 38
LI normal: 938.4904305107437 +/- 558.1397
LI pathol: 549.8706828941058 +/- 305.3874


In [10]:
# test p values for fd
# check p values agains the paper - IDENTICAL
li_t, li_p_value = stats.ttest_ind(n_li, p_li)
print("LI P-value:", fd_p_value)

#extenstion
n_li1 = []
p_li1 = []
for r in n_li_ext_flex:
    n_li1.append(r[0])
for r in p_li_ext_flex:
    p_li1.append(r[0])

li1_t, li1_p_value = stats.ttest_ind(n_li1, p_li1)
print("LI1 P-value:", li1_p_value)

n_li2 = []
p_li2 = []
for r in n_li_ext_flex:
    n_li2.append(r[1])
for r in p_li_ext_flex:
    p_li2.append(r[1])

li2_t, li2_p_value = stats.ttest_ind(n_li2, p_li2)
print("LI2 P-value:", li2_p_value)


LI P-value: 8.901377374682255e-89
LI1 P-value: 0.3635732686481704
LI2 P-value: 0.49974575970771573


In [11]:
# turn the computed features into a dataframe

df = pd.DataFrame()

# state
n_state = np.ones(len(n_res),dtype=int)
p_state = np.zeros(len(p_res), dtype=int)
state = np.concatenate((n_state, p_state))

# FD
fd = np.concatenate((n_fd, p_fd))
n_fd_ext_flx = np.array(n_ext_flex_fd)
p_fd_ext_flx = np.array(p_ext_flex_fd)
n_fd_ext = n_fd_ext_flx[:, 0]
n_fd_flx = n_fd_ext_flx[:, 1]
p_fd_ext = p_fd_ext_flx[:, 0]
p_fd_flx = p_fd_ext_flx[:, 1]

fd_ext = np.concatenate((n_fd_ext, p_fd_ext))
fd_flx = np.concatenate((n_fd_flx, p_fd_flx))


# VMS
#n_mean_sqr = [] n_mean_sqr_ext_flex = [] p_mean_sqr = [] p_mean_sqr_ext_flex = []
vms = np.concatenate((n_mean_sqr, p_mean_sqr))
n_vms1 = np.array(n_mean_sqr_ext_flex)
p_vms1 = np.array(p_mean_sqr_ext_flex)
n_vms_ext = n_vms1[:, 0]
n_vms_flx = n_vms1[:, 1]
p_vms_ext = p_vms1[:, 0]
p_vms_flx = p_vms1[:, 1]

vms_ext = np.concatenate((n_vms_ext, p_vms_ext))
vms_flx = np.concatenate((n_vms_flx, p_vms_flx))

# LI
#n_li = [] n_li_ext_flex = [] p_li = [] p_li_ext_flex = []

li = np.concatenate((n_li, p_li))
n_li1 = np.array(n_li_ext_flex)
p_li1 = np.array(p_li_ext_flex)
print(len(n_li1))
n_li_ext = n_li1[:, 0]
n_li_flx = n_li1[:, 1]
p_li_ext = p_li1[:, 0]
p_li_flx = p_li1[:, 1]

li_ext = np.concatenate((n_li_ext, p_li_ext))
li_flx = np.concatenate((n_li_flx, p_li_flx))

df["STATE"] = state
df['FD'] = fd
df['FD_EXT'] = fd_ext
df['FD_FLX'] = fd_flx
df['VMS'] = vms
df['VMS_EXT'] = vms_ext
df['VMS_FLX'] = vms_flx
df['LI'] = li
df['LI_EXT'] = li_ext
df['LI_FLX'] = li_flx

print(df.head())


51
   STATE        FD    FD_EXT    FD_FLX       VMS       VMS_EXT       VMS_FLX  \
0      1  1.714624 -1.953987 -1.893786  0.003594  7.705657e-09  5.391020e-08   
1      1  1.876535 -2.437410 -2.631023  0.017189  2.282283e-08  4.681395e-08   
2      1  1.858951 -2.675034 -2.419703  0.010199  4.082311e-08  3.568691e-08   
3      1  1.640282 -1.989119 -2.217110  0.003709  4.987345e-08  3.646279e-09   
4      1  1.743807 -2.269572 -2.401506  0.024151  3.403807e-08  7.599448e-09   

            LI      LI_EXT      LI_FLX  
0   684.049676   74.933167  103.381055  
1  1746.572715  154.270082  159.203130  
2  1919.730051  197.739256  183.806528  
3   559.070428   62.157193   73.515731  
4  1238.495077  134.842340  131.481354  


In [12]:
# write df to csv in features directory

In [13]:
directory = 'features'
file_name = 'vag_features.csv'
file_path = os.path.join(directory, file_name)

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Write the DataFrame to a CSV file
df.to_csv(file_path, index=False)  # Set index=False to avoid writing row indices